# Process NeuroQuery Data into Yale Brain Atlas Space

In [1]:
# Load packages
from neuroquery import fetch_neuroquery_model, NeuroQueryModel
from nilearn.plotting import view_img
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [10]:
# Download NeuroQuery encoder
encoder = NeuroQueryModel.from_data_dir(fetch_neuroquery_model())

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [30]:
# Functions to convert between voxel and world coordinate space; requires extracting affine matrix
def voxel_to_world_coordinates(voxel_coords, affine):
    """
    Convert voxel coordinates to world coordinates.
    :param voxel_coords: numpy array of voxel coordinates (3,)
    :param affine: numpy array representing the affine transformation (4, 4)
    :return: numpy array of world coordinates (3,)
    """
    voxel_coords_homogeneous = np.append(voxel_coords, 1)  # Convert to homogeneous coordinates
    world_coords_homogeneous = affine @ voxel_coords_homogeneous  # Matrix multiplication
    return world_coords_homogeneous[:3]  # Return only the x, y, z world coordinates

def world_to_voxel_coordinates(world_coords, affine):
    """
    Convert world coordinates to voxel indices.
    
    Parameters:
    - world_coords: numpy array of shape (3,) for world coordinates (x, y, z).
    - affine: numpy array of shape (4, 4), the affine transformation matrix.
    
    Returns:
    - voxel_coords: numpy array of shape (3,) for voxel indices.
    """
    # Invert the affine matrix
    affine_inv = np.linalg.inv(affine)
    # Convert world coordinates to homogeneous coordinates
    world_homogeneous = np.append(world_coords, 1)
    # Apply the inverted affine matrix
    voxel_homogeneous = np.dot(affine_inv, world_homogeneous)
    # Drop the homogeneous coordinate
    voxel_coords = voxel_homogeneous[:3]
    return voxel_coords

In [31]:
# Get brain maps for each of the 334 functional terms
metaanalysis_functional_terms_df = pd.read_csv("../data/metaanalysis/metaanalysis_functional_terms.csv")
functional_terms = metaanalysis_functional_terms_df["new_neuroquery_functional_term"].unique()

brain_map_affine = encoder("math")["brain_map"].affine # standard across all terms

brain_map_allterms_data = np.empty((46, 55, 46, 334)) # dimensions x,y,z,functional terms
for i in range(len(functional_terms)):
    brain_map_i = encoder(functional_terms[i])["brain_map"]
    brain_map_i_data = brain_map_i.get_fdata()
    brain_map_allterms_data[:,:,:,i] = brain_map_i_data

In [32]:
# Load Yale Brain Atlas positions (whole brain) and parcel names
atlas_whole_positions = pd.read_csv("../data/atlas/atlas_whole_positions.csv")
parcel_names = atlas_whole_positions["parcel"].unique()

In [33]:
# Now, round the 'x', 'y', and 'z' columns to the nearest integer
# Important for indexing with NeuroQuery maps
atlas_whole_positions_rounded = atlas_whole_positions
atlas_whole_positions_rounded['x'] = atlas_whole_positions_rounded['x'].round(0)
atlas_whole_positions_rounded['y'] = atlas_whole_positions_rounded['y'].round(0)
atlas_whole_positions_rounded['z'] = atlas_whole_positions_rounded['z'].round(0)

# Convert to voxel coordinates for indexing purposes
for index, row in atlas_whole_positions_rounded.iterrows():
    voxel_coords = world_to_voxel_coordinates(np.array([row['x'], row['y'], row['z']]), brain_map_affine)
    atlas_whole_positions_rounded.loc[index, ['x_voxel', 'y_voxel', 'z_voxel']] = voxel_coords

# Now, round the 'x', 'y', and 'z' columns to the nearest integer
atlas_whole_positions_rounded['x_voxel'] = atlas_whole_positions_rounded['x_voxel'].round(0).astype(int)
atlas_whole_positions_rounded['y_voxel'] = atlas_whole_positions_rounded['y_voxel'].round(0).astype(int)
atlas_whole_positions_rounded['z_voxel'] = atlas_whole_positions_rounded['z_voxel'].round(0).astype(int)

In [34]:
# For each parcel, get the activations at the constituent x,y,z positions for each term, then take average for each term
# Resulting dataframe is average activation value for 334 functional terms across 696 parcels
neuroquery_functional_activation_database = np.empty((696, 334))
for i in range(len(parcel_names)):
    parcel_i = parcel_names[i]
    atlas_whole_positions_rounded_parcel_i = atlas_whole_positions_rounded[atlas_whole_positions_rounded['parcel'] == parcel_names[i]]
    for j in range(len(functional_terms)):
        brain_map_j_data = brain_map_allterms_data[:,:,:,j]
        activation_values_parcel_i_term_j = brain_map_j_data[atlas_whole_positions_rounded_parcel_i["x_voxel"].values,
                                                             atlas_whole_positions_rounded_parcel_i["y_voxel"].values,
                                                             atlas_whole_positions_rounded_parcel_i["z_voxel"].values]
        neuroquery_functional_activation_database[i,j] = np.mean(activation_values_parcel_i_term_j)

neuroquery_functional_activation_database = pd.DataFrame(neuroquery_functional_activation_database)

In [43]:
# Rename columns to functional terms
column_name_map = dict(zip(list(neuroquery_functional_activation_database.columns.values), functional_terms))
neuroquery_functional_activation_database = neuroquery_functional_activation_database.rename(columns = column_name_map)

In [54]:
# Convert all negative activations to 0
neuroquery_functional_activation_database = neuroquery_functional_activation_database.clip(lower=0)

In [ ]:
neuroquery_functional_activation_database

In [61]:
# Save
neuroquery_functional_activation_database.to_csv("../data/neurosynth/neuroquery_functional_activation_database.csv", index = False)